**Sentiment Analysis of IMDB Movie Reviews**


This Notebook is heavily based on the Notebook by [Lakshmipathi N](https://www.kaggle.com/lakshmi25npathi) found on [Kaggle](https://www.kaggle.com/lakshmi25npathi/sentiment-analysis-of-imdb-movie-reviews).

**Import necessary libraries**

In [1]:
#Load the libraries
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
# https://online.stat.psu.edu/stat504/lesson/1/1.7
from utils import preprocesser_text, binarize_sentiment, train_test_split, evaluate

import os
import warnings

**Import the training dataset**

In [2]:
#importing the training data
imdb_data=pd.read_csv('data/IMDB Dataset.csv')
print(imdb_data.shape)
imdb_data.head(10)

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


**Exploratery data analysis**

In [3]:
#Summary of the dataset
imdb_data.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


**Sentiment count**

In [4]:
#sentiment count
imdb_data['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

We can see that the dataset is balanced.

**Spliting the training dataset**

In [5]:
imdb_data_norm = preprocesser_text(imdb_data)

**Text normalization**
When doing a very basic train/test-split, we should be sure that we have close to a 50/50 Balance of Classes in the Train-Test set. This is the case here.

In [6]:
#normalized train reviews
norm_train, norm_test = train_test_split(imdb_data_norm)
print(norm_train.sentiment.value_counts())
print(norm_test.sentiment.value_counts())
norm_train_reviews=norm_train.review
norm_train_reviews[0]

negative    20007
positive    19993
Name: sentiment, dtype: int64
positive    5007
negative    4993
Name: sentiment, dtype: int64


'one review ha mention watch 1 oz episod youll hook right thi exactli happen meth first thing struck oz wa brutal unflinch scene violenc set right word go trust thi show faint heart timid thi show pull punch regard drug sex violenc hardcor classic use wordit call oz nicknam given oswald maximum secur state penitentari focus mainli emerald citi experiment section prison cell glass front face inward privaci high agenda em citi home manyaryan muslim gangsta latino christian italian irish moreso scuffl death stare dodgi deal shadi agreement never far awayi would say main appeal show due fact goe show wouldnt dare forget pretti pictur paint mainstream audienc forget charm forget romanceoz doesnt mess around first episod ever saw struck nasti wa surreal couldnt say wa readi watch develop tast oz got accustom high level graphic violenc violenc injustic crook guard wholl sold nickel inmat wholl kill order get away well manner middl class inmat turn prison bitch due lack street skill prison exp

**Bags of words model**

It is used to convert text documents to numerical vectors. It also creates n_grams, which basically means that if n = one, one word == 1 vector. If n = 2, a vector is made up of two words etc. One row then is equal to how often a specific n_gram appears in the, in this case, Review.

In [7]:
#Count vectorizer for bag of words
cv=CountVectorizer(ngram_range=(1,3))
#transformed train reviews
cv_train_reviews=cv.fit_transform(norm_train.review)
#transformed test reviews
cv_test_reviews=cv.transform(norm_test.review)

print('BOW_cv_train:',cv_train_reviews.shape)
print('BOW_cv_test:',cv_test_reviews.shape)
print('Vocab: ', cv.get_feature_names()[:5])
print('Vocab Length: ', len(cv.get_feature_names()))

BOW_cv_train: (40000, 6983231)
BOW_cv_test: (10000, 6983231)
Vocab:  ['00', '00 1991', '00 1991 wa', '00 act', '00 act charact']
Vocab Length:  6983231


In [8]:
print('Vocab: ', len(cv.get_feature_names()))

Vocab:  6983231


In [9]:
cv_train_reviews[0]

<1x6983231 sparse matrix of type '<class 'numpy.int64'>'
	with 486 stored elements in Compressed Sparse Row format>

**Term Frequency-Inverse Document Frequency model (TFIDF)**

It is used to convert text documents to  matrix of  tfidf features. Basically, per review it will calculate how often a word appears in this review and multiples it with the inverse document frequency, which is basically a "punishing" value if a word appears in many documents. Basically: # number of documents / # documents containing the word. 

> The formula that is used to compute the tf-idf for a term t of a document d in a document set is tf-idf(t, d) = tf(t, d) * idf(t), and the idf is computed as idf(t) = log [ n / df(t) ] + 1 (if smooth_idf=False), where n is the total number of documents in the document set and df(t) is the document frequency of t; the document frequency is the number of documents in the document set that contain the term t. The effect of adding “1” to the idf in the equation above is that terms with zero idf, i.e., terms that occur in all documents in a training set, will not be entirely ignored. (Note that the idf formula above differs from the standard textbook notation that defines the idf as idf(t) = log [ n / (df(t) + 1) ]).

tf(t, d) == # Wort t / max (# Wort t) über alle Dokumente

idf(t) == # Dokumente mit Wort t / # Dokumente


tf-idf(t, d) = tf(t, d) * idf(t)

In [11]:
#Tfidf vectorizer
tv=TfidfVectorizer(ngram_range=(1,3))
#transformed train reviews
tv_train_reviews=tv.fit_transform(norm_train.review)
#transformed test reviews
tv_test_reviews=tv.transform(norm_test.review)
print('Tfidf_train:',tv_train_reviews.shape)
print('Tfidf_test:',tv_test_reviews.shape)
print('Tfidf_test:',len(tv.vocabulary_))

Tfidf_train: (40000, 6983231)
Tfidf_test: (10000, 6983231)
Tfidf_test: 6983231


**Split and binarize the sentiment tdata**

In [12]:
#Spliting the sentiment data
train_sentiments=norm_train.sentiment
test_sentiments=norm_test.sentiment

test_sentiments = binarize_sentiment(test_sentiments)
train_sentiments = binarize_sentiment(train_sentiments)
print(train_sentiments)
print(test_sentiments)

0        1
1        1
2        1
3        0
4        1
        ..
39995    1
39996    1
39997    1
39998    0
39999    0
Name: sentiment, Length: 40000, dtype: int64
40000    0
40001    0
40002    0
40003    0
40004    0
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 10000, dtype: int64


**Modelling the dataset**

Let us build logistic regression model for both bag of words and tfidf features and do a quick parameter-search to avoid heavy overfitting.

In [13]:
from sklearn.model_selection import GridSearchCV

In [14]:
#training the model
lr=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=42)
# Gridsearch to find the best values:
parameters = [{'C': [1,10,50,100]}]
grid_search_cv = GridSearchCV(estimator = lr,
                          param_grid = parameters, 
                          scoring = 'accuracy',
                          cv = 5,
                          n_jobs = -1, 
                          verbose = 2)
grid_search_cv.fit(cv_train_reviews,train_sentiments)
print(f'Best Score: {grid_search_cv.best_score_}')

# Tfidf
grid_search_tfidf = GridSearchCV(estimator = lr,
                          param_grid = parameters, 
                          scoring = 'accuracy',
                          cv = 5,
                          n_jobs = -1,
                          verbose = 2)
grid_search_tfidf.fit(tv_train_reviews,train_sentiments)
print(f'Best Score: {grid_search_tfidf.best_score_}')


Fitting 5 folds for each of 4 candidates, totalling 20 fits


In [ ]:
#Fitting the model for Bag of words
lr_bow=grid_search_cv.best_estimator
print(lr_bow)
#Fitting the model for tfidf features
lr_tfidf=grid_search_tfidf.best_estimator
print(lr_tfidf)

**Logistic regression model performane on test dataset**

In [ ]:
#Predicting the model for bag of words
lr_bow_predict=lr.predict(cv_test_reviews)
print(lr_bow_predict)
##Predicting the model for tfidf features
lr_tfidf_predict=lr.predict(tv_test_reviews)
print(lr_tfidf_predict)

#Predicting the model for bag of words
lr_bow_predict_train=lr.predict(cv_train_reviews)
##Predicting the model for tfidf features
lr_tfidf_predict_train=lr.predict(tv_train_reviews)


[0 0 0 ... 0 1 1]
[0 0 0 ... 0 1 1]


**Accuracy of the model**

In [ ]:
#Accuracy score for bag of words
print("lr_bow_score test:",evaluate(test_sentiments,lr_bow_predict)[0])
#Accuracy score for tfidf features
print("lr_tfidf_score test:",evaluate(test_sentiments,lr_tfidf_predict)[0])
#Accuracy score for bag of words
print("lr_bow_score train:",evaluate(train_sentiments,lr_bow_predict_train)[0])
#Accuracy score for tfidf features
print("lr_tfidf_score train:",evaluate(train_sentiments,lr_tfidf_predict_train)[0])

lr_bow_score test: 0.7512
lr_tfidf_score test: 0.75
lr_bow_score train: 0.996275
lr_tfidf_score train: 0.996275


**Print the classification report**

In [ ]:
#Classification report for bag of words 
print(evaluate(test_sentiments,lr_bow_predict)[1])

#Classification report for tfidf features
print(print(evaluate(test_sentiments,lr_tfidf_predict)[1]))

              precision    recall  f1-score   support

    Negative       0.75      0.75      0.75      4993
    Positive       0.75      0.75      0.75      5007

    accuracy                           0.75     10000
   macro avg       0.75      0.75      0.75     10000
weighted avg       0.75      0.75      0.75     10000

              precision    recall  f1-score   support

    Negative       0.74      0.77      0.75      4993
    Positive       0.76      0.73      0.75      5007

    accuracy                           0.75     10000
   macro avg       0.75      0.75      0.75     10000
weighted avg       0.75      0.75      0.75     10000

None


**Stochastic gradient descent or Linear support vector machines for bag of words and tfidf features**

In [ ]:
#training the model
svm=SGDClassifier(loss='hinge',max_iter=2000,random_state=42)
# Gridsearch to find the best values:
parameters = [{'alpha': [0.0001, 0.001, 0.01, 0.1, 1]}]
grid_search_svm_cv = GridSearchCV(estimator = lr,
                            param_grid = parameters, 
                            scoring = 'accuracy',
                            cv = 5,
                            n_jobs = -1, 
                            verbose = 2)
grid_search_svm_cv.fit(cv_train_reviews,train_sentiments)
print(f'Best Score: {grid_search_svm_cv.best_score_}')

# Tfidf
grid_search_svm_tfidf = GridSearchCV(estimator = lr,
                            param_grid = parameters, 
                            scoring = 'accuracy',
                            cv = 5,
                            n_jobs = -1,
                            verbose = 2)
grid_search_svm_tfidf.fit(tv_train_reviews,train_sentiments)
print(f'Best Score: {grid_search_svm_tfidf.best_score_}')


In [ ]:
#fitting the svm for bag of words
svm_bow=grid_search_svm_cv.best_estimator_
print(svm_bow)
#fitting the svm for tfidf features
svm_tfidf=grid_search_svm_tfidf.best_estimator_
print(svm_tfidf)

SGDClassifier(max_iter=2000, random_state=42)
SGDClassifier(max_iter=2000, random_state=42)


**Model performance on test data**

In [ ]:
#Predicting the model for bag of words
svm_bow_predict=svm.predict(cv_test_reviews)
print(svm_bow_predict)
#Predicting the model for tfidf features
svm_tfidf_predict=svm.predict(tv_test_reviews)
print(svm_tfidf_predict)

#Predicting the model for bag of words
svm_bow_predict_train=svm.predict(cv_train_reviews)
##Predicting the model for tfidf features
svm_tfidf_predict_train=svm.predict(tv_train_reviews)

[1 1 0 ... 1 1 1]
[1 1 1 ... 1 1 1]


**Accuracy of the model**

In [ ]:
#Accuracy score for bag of words
print("svm_bow_score test:",evaluate(test_sentiments,svm_bow_predict)[0])
#Accuracy score for tfidf features
print("svm_tfidf_score test:",evaluate(test_sentiments,svm_tfidf_predict)[0])

#Accuracy score for bag of words
print("svm_bow_score train:",evaluate(train_sentiments,svm_bow_predict_train)[0])
#Accuracy score for tfidf features
print("svm_tfidf_score train:",evaluate(train_sentiments,svm_tfidf_predict_train)[0])

svm_bow_score test: 0.5829
svm_tfidf_score test: 0.5112
svm_bow_score train: 0.990425
svm_tfidf_score train: 0.990425


**Print the classification report**

In [ ]:
#Classification report for bag of words 
print(evaluate(test_sentiments,svm_bow_predict)[1])
#Classification report for tfidf features
print(evaluate(test_sentiments,svm_tfidf_predict)[1])

              precision    recall  f1-score   support

    Negative       0.94      0.18      0.30      4993
    Positive       0.55      0.99      0.70      5007

    accuracy                           0.58     10000
   macro avg       0.74      0.58      0.50     10000
weighted avg       0.74      0.58      0.50     10000

              precision    recall  f1-score   support

    Negative       1.00      0.02      0.04      4993
    Positive       0.51      1.00      0.67      5007

    accuracy                           0.51     10000
   macro avg       0.75      0.51      0.36     10000
weighted avg       0.75      0.51      0.36     10000



**Multinomial Naive Bayes for bag of words and tfidf features**

In [ ]:
#training the model
mnb=MultinomialNB()
#fitting the svm for bag of words
mnb_bow=mnb.fit(cv_train_reviews,train_sentiments)
print(mnb_bow)
#fitting the svm for tfidf features
mnb_tfidf=mnb.fit(tv_train_reviews,train_sentiments)
print(mnb_tfidf)

MultinomialNB()
MultinomialNB()


**Model performance on test data**

In [ ]:
#Predicting the model for bag of words
mnb_bow_predict=mnb.predict(cv_test_reviews)
print(mnb_bow_predict)
#Predicting the model for tfidf features
mnb_tfidf_predict=mnb.predict(tv_test_reviews)
print(mnb_tfidf_predict)

#Predicting the model for bag of words
mnb_bow_predict_train=mnb.predict(cv_train_reviews)
##Predicting the model for tfidf features
mnb_tfidf_predict_train=mnb.predict(tv_train_reviews)

[0 0 0 ... 0 1 1]
[0 0 0 ... 0 1 1]


**Accuracy of the model**

In [ ]:
#Accuracy score for bag of words
print("mnb_bow_score test:",evaluate(test_sentiments,mnb_bow_predict)[0])
print("mnb_bow_score train:",evaluate(train_sentiments,mnb_bow_predict_train)[0])
#Accuracy score for tfidf features
print("mnb_tfidf_score test:",evaluate(test_sentiments,mnb_tfidf_predict)[0])
print("mnb_tfidf_score train:",evaluate(train_sentiments,mnb_tfidf_predict_train)[0])

mnb_bow_score test: 0.751
mnb_bow_score train: 0.996275
mnb_tfidf_score test: 0.7509
mnb_tfidf_score train: 0.996275


**Print the classification report**

In [ ]:
#Classification report for bag of words 
print(evaluate(test_sentiments,mnb_bow_predict)[1])
#Classification report for tfidf features
print(evaluate(test_sentiments,mnb_tfidf_predict)[1])